In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
data = pd.read_csv("./stocks_data/AAPL_data.csv")
data.head()

,Date,Open,High,Low,Close,Volume,EMA_12,EMA_26,MACD,Signal,RSI,CCI,ADX
0,2015-03-02 00:00:00-05:00,28.865108,29.095135,28.652947,28.829374,192386800,28.829374,28.829374,0.000000,0.000000,45.526105,56.240216,28.673114
1,2015-03-03 00:00:00-05:00,28.800338,28.925401,28.606040,28.889668,151265200,28.838650,28.833840,0.004810,0.000962,45.526105,56.240216,28.673114
2,2015-03-04 00:00:00-05:00,28.831606,28.934334,28.657410,28.706539,126665200,28.818325,28.824411,-0.006085,-0.000448,45.526105,56.240216,28.673114
3,2015-03-05 00:00:00-05:00,28.715477,28.753442,28.085693,28.230856,226068400,28.727945,28.780444,-0.052498,-0.010858,45.526105,56.240216,28.673114
4,2015-03-06 00:00:00-05:00,28.675274,28.891902,28.197354,28.273285,291368400,28.657998,28.742876,-0.084879,-0.025662,45.526105,56.240216,28.673114


In [3]:
len(data), data.shape

(2515, (2515, 13))

In [4]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'EMA_12', 'EMA_26',
       'MACD', 'Signal', 'RSI', 'CCI', 'ADX'],
      dtype='object')

In [5]:
data = data.set_index("Date")

In [6]:
data.tail()

,Open,High,Low,Close,Volume,EMA_12,EMA_26,MACD,Signal,RSI,CCI,ADX
Date,,,,,,,,,,,,
2025-02-21 00:00:00-05:00,245.949997,248.690002,245.220001,245.550003,53197400,240.208448,237.920002,2.288446,0.363129,62.766179,103.163767,31.180457
2025-02-24 00:00:00-05:00,244.929993,248.860001,244.419998,247.100006,51326400,241.268688,238.600002,2.668686,0.824240,80.231290,90.587796,28.337470
2025-02-25 00:00:00-05:00,248.000000,250.000000,244.910004,247.039993,48013300,242.156581,239.225187,2.931394,1.245671,76.585168,83.906297,24.582475
2025-02-26 00:00:00-05:00,244.330002,244.979996,239.130005,240.360001,44433600,241.880184,239.309247,2.570937,1.510724,62.116289,19.244203,21.212065
2025-02-27 00:00:00-05:00,239.410004,242.460007,237.059998,237.300003,41078200,241.175541,239.160414,2.015127,1.611605,56.035173,-15.982410,18.992802


In [7]:
# normalizing data
data = data.values
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled[0]

array([0.03484219, 0.03398254, 0.03452298, 0.03425526, 0.2703879 ,
       0.03184143, 0.03052919, 0.42180037, 0.40620287, 0.45542002,
       0.59169241, 0.3071847 ])

In [8]:
# preparing dataset for LSTM

sequence_len = 30   #considering 30 days window
xs = []
ys = []

for i in range(len(data_scaled) - sequence_len):
    xs.append(data_scaled[i:i+sequence_len])
    ys.append(data_scaled[i+sequence_len])

x = torch.tensor(xs).type(torch.float32)
y = torch.tensor(ys).type(torch.float32)
x[0], y[0]

C:\conda_tmp\ipykernel_1912\1879639991.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  x = torch.tensor(xs).type(torch.float32)


(tensor([[0.0348, 0.0340, 0.0345, 0.0343, 0.2704, 0.0318, 0.0305, 0.4218, 0.4062,
          0.4554, 0.5917, 0.3072],
         [0.0346, 0.0333, 0.0343, 0.0345, 0.2047, 0.0319, 0.0305, 0.4221, 0.4063,
          0.4554, 0.5917, 0.3072],
         [0.0347, 0.0333, 0.0345, 0.0337, 0.1653, 0.0318, 0.0305, 0.4214, 0.4062,
          0.4554, 0.5917, 0.3072],
         [0.0342, 0.0326, 0.0321, 0.0317, 0.3242, 0.0314, 0.0303, 0.4184, 0.4054,
          0.4554, 0.5917, 0.3072],
         [0.0340, 0.0331, 0.0326, 0.0319, 0.4286, 0.0311, 0.0301, 0.4163, 0.4044,
          0.4554, 0.5917, 0.3072],
         [0.0336, 0.0333, 0.0315, 0.0324, 0.5289, 0.0309, 0.0300, 0.4153, 0.4033,
          0.4554, 0.5917, 0.3072],
         [0.0322, 0.0311, 0.0303, 0.0300, 0.4031, 0.0304, 0.0297, 0.4116, 0.4016,
          0.4554, 0.5917, 0.3072],
         [0.0306, 0.0288, 0.0287, 0.0278, 0.4037, 0.0296, 0.0293, 0.4062, 0.3991,
          0.4554, 0.5917, 0.3072],
         [0.0283, 0.0290, 0.0282, 0.0299, 0.2721, 0.0292, 0.0290

In [9]:
x.shape, y.shape

(torch.Size([2485, 30, 12]), torch.Size([2485, 12]))

In [10]:
# dividing data into train and test set
split_ratio = 0.8
split_size = int(len(x) * split_ratio)
print(split_size)

X_train = x[:split_size]
X_test = x[split_size:]
y_train = y[:split_size]
y_test = y[split_size:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

1988
torch.Size([1988, 30, 12]) torch.Size([497, 30, 12]) torch.Size([1988, 12]) torch.Size([497, 12])


In [11]:
X_train[0], y_train[0]

(tensor([[0.0348, 0.0340, 0.0345, 0.0343, 0.2704, 0.0318, 0.0305, 0.4218, 0.4062,
          0.4554, 0.5917, 0.3072],
         [0.0346, 0.0333, 0.0343, 0.0345, 0.2047, 0.0319, 0.0305, 0.4221, 0.4063,
          0.4554, 0.5917, 0.3072],
         [0.0347, 0.0333, 0.0345, 0.0337, 0.1653, 0.0318, 0.0305, 0.4214, 0.4062,
          0.4554, 0.5917, 0.3072],
         [0.0342, 0.0326, 0.0321, 0.0317, 0.3242, 0.0314, 0.0303, 0.4184, 0.4054,
          0.4554, 0.5917, 0.3072],
         [0.0340, 0.0331, 0.0326, 0.0319, 0.4286, 0.0311, 0.0301, 0.4163, 0.4044,
          0.4554, 0.5917, 0.3072],
         [0.0336, 0.0333, 0.0315, 0.0324, 0.5289, 0.0309, 0.0300, 0.4153, 0.4033,
          0.4554, 0.5917, 0.3072],
         [0.0322, 0.0311, 0.0303, 0.0300, 0.4031, 0.0304, 0.0297, 0.4116, 0.4016,
          0.4554, 0.5917, 0.3072],
         [0.0306, 0.0288, 0.0287, 0.0278, 0.4037, 0.0296, 0.0293, 0.4062, 0.3991,
          0.4554, 0.5917, 0.3072],
         [0.0283, 0.0290, 0.0282, 0.0299, 0.2721, 0.0292, 0.0290

In [12]:
### build LSTM model

class LSTMv0(nn.Module):
    def __init__(self, input_features, hidden_features, num_layers, output_features):
        super().__init__()
        self.hidden_features = hidden_features
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_features, hidden_features, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_features, output_features)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_features)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_features)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out[:, -1, :]

In [13]:
# parameters
INPUT_FEATURES = len(X_train[0, 0, :])
HIDDEN_FEATURES = 30
NUM_LAYERS = 3
OUTPUT_FEATURES = len(y_train[0, :])

model_0 = LSTMv0(INPUT_FEATURES, HIDDEN_FEATURES, NUM_LAYERS, OUTPUT_FEATURES)

In [14]:
model_0

LSTMv0(
  (lstm): LSTM(12, 30, num_layers=3, batch_first=True)
  (fc): Linear(in_features=30, out_features=12, bias=True)
)

In [15]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01)

In [17]:
# training loop
epochs = 200

for epoch in range(epochs):
    # train mode
    model_0.train()

    # forward pass
    y_pred = model_0(X_train)

    # calulate loss
    loss = loss_fn(y_pred, y_train)

    # optimizer zero grad
    optimizer.zero_grad()

    # backward
    loss.backward()

    # step optimizer
    optimizer.step()

    model_0.eval()
    with torch.inference_mode():
        test_pred = model_0(X_test)
        test_loss = loss_fn(test_pred, y_test)

    # print
    if epoch % 20 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.4f} | Test Loss: {test_loss:.4f}")

Epoch: 0 | Loss: 0.1438 | Test Loss: 0.3873
Epoch: 20 | Loss: 0.1343 | Test Loss: 0.3722
Epoch: 40 | Loss: 0.1255 | Test Loss: 0.3583
Epoch: 60 | Loss: 0.1176 | Test Loss: 0.3453
Epoch: 80 | Loss: 0.1102 | Test Loss: 0.3332
Epoch: 100 | Loss: 0.1036 | Test Loss: 0.3219
Epoch: 120 | Loss: 0.0974 | Test Loss: 0.3113
Epoch: 140 | Loss: 0.0918 | Test Loss: 0.3014
Epoch: 160 | Loss: 0.0867 | Test Loss: 0.2922
Epoch: 180 | Loss: 0.0819 | Test Loss: 0.2836


In [18]:
# evaluate model with test values
model_0.eval()
with torch.inference_mode():
    eval_pred = model_0(X_test)
eval_pred[:5]

tensor([[ 0.1817,  0.0841,  0.0656,  0.0115,  0.0249,  0.1987, -0.0058,  0.1386,
          0.2414,  0.3063,  0.2020,  0.0855],
        [ 0.1817,  0.0841,  0.0656,  0.0114,  0.0248,  0.1989, -0.0059,  0.1386,
          0.2412,  0.3063,  0.2020,  0.0854],
        [ 0.1817,  0.0840,  0.0657,  0.0113,  0.0247,  0.1991, -0.0059,  0.1386,
          0.2412,  0.3063,  0.2020,  0.0853],
        [ 0.1818,  0.0840,  0.0657,  0.0112,  0.0247,  0.1992, -0.0060,  0.1386,
          0.2412,  0.3062,  0.2020,  0.0853],
        [ 0.1818,  0.0840,  0.0656,  0.0112,  0.0247,  0.1993, -0.0060,  0.1386,
          0.2412,  0.3063,  0.2020,  0.0853]])

In [20]:
eval_pred[-5: ]

tensor([[ 0.1838,  0.0838,  0.0634,  0.0108,  0.0244,  0.2005, -0.0067,  0.1374,
          0.2425,  0.3060,  0.2017,  0.0864],
        [ 0.1839,  0.0837,  0.0633,  0.0108,  0.0245,  0.2004, -0.0067,  0.1373,
          0.2428,  0.3058,  0.2015,  0.0864],
        [ 0.1839,  0.0836,  0.0634,  0.0107,  0.0245,  0.2004, -0.0067,  0.1372,
          0.2430,  0.3055,  0.2014,  0.0864],
        [ 0.1840,  0.0835,  0.0635,  0.0106,  0.0245,  0.2004, -0.0066,  0.1371,
          0.2432,  0.3053,  0.2012,  0.0864],
        [ 0.1840,  0.0834,  0.0635,  0.0105,  0.0246,  0.2004, -0.0066,  0.1370,
          0.2434,  0.3051,  0.2011,  0.0865]])

### **Start from scratch**

**Plan**
1. Make LSTM function that can be used to fit different dataframes, and Train in seperately
2. Make Stock Trading Env
3. Use Stable_Baselines3 to import PPO model, and Trian it with LSTM(that are trained before)

In [1]:
# ## Make a function that can take data and break-down it into different dataframes (or inputs)

# import pandas as pd
# import torch
# import torch.nn.functional as F
# from sklearn.preprocessing import MinMaxScaler

# def data_fragmenter(df: pd.DataFrame):
#     scaler = MinMaxScaler()
#     df_fundamental = df[["Open", "High", "Low", "Close", "Volume"]]
#     df_mavg = df[["EMA_12", "EMA_26"]]
#     df_mi = df[["MACD", "Signal", "RSI", "CCI"]]
#     df_adx = df[["ADX"]]

#     # normalize
#     df_fundamental = scaler.fit_transform(df_fundamental)
#     df_mavg = scaler.fit_transform(df_mavg)
#     df_mi = scaler.fit_transform(df_mi)
#     df_adx = scaler.fit_transform(df_adx)

#     return df_fundamental, df_mavg, df_mi, df_adx

# df = pd.read_csv("./stocks_data/AAPL_data.csv")
# fundamental, mavg, mi, adx = data_fragmenter(df=df)
# fundamental[:5], mavg[:5], mi[:5], adx[:5]

Break Down data into columns or dataframes
1. Open, High, Low, Close, Volume -> Fundamental Data
2. EMA_12, EMA_26 -> Moving Avgs
3. MACD, Signal, RSI, CCI -> Momentum Indicators
4. ADX -> Trend Strength (ADX)

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [2]:
import torch
import numpy as np

def data_sequencer(data: torch.Tensor, sequence_len: int, device="cpu"):
    xs = []
    ys = []

    for i in range(len(data) - sequence_len):
        xs.append(data[i: i+sequence_len])
        ys.append(data[i+sequence_len])

    xs = np.array(xs)
    ys = np.array(ys)

    xs = torch.tensor(xs, dtype=torch.float32, device=device)
    ys = torch.tensor(ys, dtype=torch.float32, device=device)

    return xs, ys


In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

def data_fragmenter(df: pd.DataFrame, device="cpu"):
    scaler = MinMaxScaler()

    df_fundamental = df[["Open", "High", "Low", "Close", "Volume"]]
    df_mavg = df[["EMA_12", "EMA_26"]]
    df_mi = df[["MACD", "Signal", "RSI", "CCI"]]
    df_adx = df[["ADX"]]

    # normalize
    df_fundamental = scaler.fit_transform(df_fundamental)
    df_mavg = scaler.fit_transform(df_mavg)
    df_mi = scaler.fit_transform(df_mi)
    df_adx = scaler.fit_transform(df_adx)

    fundamental_x, fundamental_y = data_sequencer(df_fundamental, 30, device=device)
    mavg_x, mavg_y = data_sequencer(df_mavg, 30, device=device)
    mi_x, mi_y = data_sequencer(df_mi, 30, device=device)
    adx_x, adx_y = data_sequencer(df_adx, 30, device=device)

    return fundamental_x, fundamental_y, mavg_x, mavg_y, mi_x, mi_y, adx_x, adx_y


df = pd.read_csv("./stocks_data/AAPL_data.csv")
fundamental_x, fundamental_y, mavg_x, mavg_y, mi_x, mi_y, adx_x, adx_y = data_fragmenter(df=df, device=device)


In [4]:
## Make LSTM that can adapt to shape of data
import torch
from torch import nn

class LSTMv1(nn.Module):
    def __init__(self, input_features, hidden_features, num_layers, output_features, device="cpu"):
        super().__init__()
        self.hidden_features = hidden_features
        self.num_layers = num_layers
        self.device = device

        self.lstm = nn.LSTM(input_features, hidden_features, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_features, output_features)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_features, device=self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_features, device=self.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out[:, -1, :]


In [ ]:
# INPUT_FEATURES = len(X_train[0, 0, :])
# HIDDEN_FEATURES = 30
# NUM_LAYERS = 3
# OUTPUT_FEATURES = len(y_train[0, :])

model_fundamental = LSTMv1(len(fundamental_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(fundamental_y[0, :]), device=device)
model_mavg = LSTMv1(len(mavg_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(mavg_y[0, :]), device=device)
model_mi = LSTMv1(len(mi_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(mi_y[0, :]), device=device)
model_adx = LSTMv1(len(adx_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(adx_y[0, :]), device=device)


In [6]:
model_fundamental, model_mavg, model_mi, model_adx

(LSTMv1(
   (lstm): LSTM(5, 30, num_layers=3, batch_first=True)
   (fc): Linear(in_features=30, out_features=5, bias=True)
 ),
 LSTMv1(
   (lstm): LSTM(2, 30, num_layers=3, batch_first=True)
   (fc): Linear(in_features=30, out_features=2, bias=True)
 ),
 LSTMv1(
   (lstm): LSTM(4, 30, num_layers=3, batch_first=True)
   (fc): Linear(in_features=30, out_features=4, bias=True)
 ),
 LSTMv1(
   (lstm): LSTM(1, 30, num_layers=3, batch_first=True)
   (fc): Linear(in_features=30, out_features=1, bias=True)
 ))

In [7]:
fundamental_loss_fn = nn.MSELoss().to(device)
mavg_loss_fn = nn.MSELoss().to(device)
mi_loss_fn = nn.MSELoss().to(device)
adx_loss_fn = nn.MSELoss().to(device)

optimizer_fundamental = torch.optim.Adam(params=model_fundamental.parameters(), lr=0.001)
optimizer_mavg = torch.optim.Adam(params=model_mavg.parameters(), lr=0.001)
optimizer_mi = torch.optim.Adam(params=model_mi.parameters(), lr=0.001)
optimizer_adx = torch.optim.Adam(params=model_adx.parameters(), lr=0.001)

In [8]:
## Alternate training
import random
import os
file = random.choice(os.listdir("./stocks_data"))
file

'MRK_data.csv'

1. Pick a random file
2. read csv data as df
3. process df with data_fragmenter()
4. provide inputs to model
5. train certain epochs(10) with data

In [ ]:
import random, os, fnmatch
import pandas as pd
import torch
epochs = 200

for epoch in range(epochs):
    # pick a random file from dir
    file = random.choice(os.listdir("./stocks_data"))
    if file.endswith(".csv"):
        # read csv file
        df = pd.read_csv(file)
        fundamental_x, fundamental_y, mavg_x, mavg_y, mi_x, mi_y, adx_x, adx_y = data_fragmenter(df=df, device=device)

        ## training loop


30
